In [ ]:
#任务简述：
#目标是构建一个分类模型，预测企业客户是否会流失（churned: 0/1），属于典型的二元分类问题。
#数据包含：公司规模、使用行为、合同类型、定价、活动数据等特征，都是和B2B软件使用行为相关的指标。

#目前重点：
#任务类型：二分类
#目标变量：churned
#特征类型：有数值型、分类型（如Yes/No, plan_type, region等）、比例型、时间型



In [ ]:
# 1. Explain your data preparation process. How did you treat the data to prepare it for modelling?
# 2. Which models did you choose to evaluate, and why? Provide the supporting data analysis used to inform your choice, and explain why the models you selected are expected to work well for the problem.
# 3. Make a final recommendation on a model. Discuss why this is your recommendation, and the trade-offs you've considered in your decision. 
# 4. Discuss what techniques you used to ensure you have built the best version of the model possible, given the data you have.

In [ ]:
import pandas as pd

df = pd.read_csv('data_file.csv')
